# EDA


- The os module has a perfect method to list files in a directory.
- Pandas json normalize could work here but is not necessary to convert the JSON data to a dataframe.
- You may need a nested for-loop to access each sale!
- We've put a lot of time into creating the structure of this repository, and it's a good example for future projects.  In the file functions_variables.py, there is an example function that you can import and use.  If you have any variables, functions or classes that you want to make, they can be put in the functions_variables.py file and imported into a notebook.  Note that only .py files can be imported into a notebook. If you want to import everything from a .py file, you can use the following:
```python
from functions_variables import *
```
If you just import functions_variables, then each object from the file will need to be prepended with "functions_variables"\
Using this .py file will keep your notebooks very organized and make it easier to reuse code between notebooks.

In [56]:
# (this is not an exhaustive list of libraries)
import pandas as pd
import numpy as np
import os
import json
from pprint import pprint

## Data Importing

In [57]:
# load one file first to see what type of data you're dealing with and what attributes it has

In [58]:
# loop over all files and put them into a dataframe
from functions_importing import get_data


In [59]:
filepath = r"C:\Users\mebar\Documents\LHL-DataSci\Projects\midterm_project_housing_prices\Data"

In [60]:
raw_data = get_data(filepath)
raw_data.head()

,tags,status,list_date,open_houses,descrip_year_built,descrip_baths_3qtr,descrip_sold_date,descrip_sold_price,descrip_baths_full,descrip_name,...,flags_is_foreclosure,flags_is_plan,flags_is_coming_soon,flags_is_new_listing,community,products,listing_id,price_reduced_amount,location_state,location_city
0,"[carport, community_outdoor_space, cul_de_sac,...",sold,2023-06-29T21:16:25.000000Z,None,1963.0,NaN,2023-09-18,NaN,2.0,None,...,None,None,None,False,None,{'brand_name': 'basic_opt_in'},2957241843,45000.0,Alaska,Juneau
1,None,sold,None,None,NaN,NaN,2023-08-22,NaN,NaN,None,...,None,None,None,None,None,None,None,NaN,Alaska,Juneau
2,None,sold,None,None,NaN,NaN,2023-08-22,NaN,NaN,None,...,None,None,None,None,None,None,None,NaN,Alaska,Juneau
3,None,sold,None,None,NaN,NaN,2023-08-21,NaN,NaN,None,...,None,None,None,None,None,None,None,NaN,Alaska,Juneau
4,None,sold,None,None,NaN,NaN,2023-08-21,NaN,NaN,None,...,None,None,None,None,None,None,None,NaN,Alaska,Juneau


## Data Cleaning and Wrangling

At this point, ensure that you have all sales in a dataframe.
- Take a quick look at your data (i.e. `.info()`, `.describe()`) - what do you see?
- Is each cell one value, or do some cells have lists?
- What are the data types of each column?
- Some sales may not actually include the sale price (target).  These rows should be dropped.
- There are a lot of NA/None values.  Should these be dropped or replaced with something?
    - You can drop rows or use various methods to fills NA's - use your best judgement for each column 
    - i.e. for some columns (like Garage), NA probably just means no Garage, so 0
- Drop columns that aren't needed
    - Don't keep the list price because it will be too close to the sale price. Assume we want to predict the price of houses not yet listed

In [61]:
from functions_importing import extract_tags

In [62]:
with_tags = extract_tags(raw_data)

In [63]:
data = with_tags.copy()
data.shape

(8159, 194)

Empty columns: 

In [64]:
data.loc[:,['descrip_name','descrip_baths_1qtr','flags_is_new_construction','flags_is_for_rent','flags_is_subdivision','flags_is_contingent',
            'flags_is_pending','flags_is_plan','flags_is_coming_soon','open_houses']].isna().value_counts()

descrip_name  descrip_baths_1qtr  flags_is_new_construction  flags_is_for_rent  flags_is_subdivision  flags_is_contingent  flags_is_pending  flags_is_plan  flags_is_coming_soon
True          True                True                       True               True                  True                 True              True           True                    8159
Name: count, dtype: int64

In [65]:
# drop empty columns, old tags column
data = data.drop(['descrip_name','descrip_baths_1qtr','flags_is_new_construction','flags_is_for_rent','flags_is_subdivision','flags_is_contingent',
            'flags_is_pending','flags_is_plan','flags_is_coming_soon','tags'],axis=1)

'community', 'flags_is_foreclosure', 'courtyard_entry', 'well_water' and 'open_house' are basically empty; flags_is_new_listing and status have all the same value, so drop them as well.

In [66]:
print('community: rows with data:',len(data[~data['community'].isna()]))
data['flags_is_new_listing'].value_counts()

community: rows with data: 5


flags_is_new_listing
False    7752
Name: count, dtype: int64

In [ ]:
data = data.drop(['community','flags_is_foreclosure','flags_is_new_listing','status','open_house','courtyard_entry','well_water'],axis=1)

None of these columns contain anything that's interesting for regression, so they are being dropped.

In [116]:
display(data['products'].value_counts())
display(data['lead_attributes'].value_counts())

products
{'brand_name': 'essentials'}         4869
{'brand_name': 'basic_opt_in'}       2176
{'brand_name': 'advantage_brand'}     587
{'brand_name': 'advantage_pro'}        10
Name: count, dtype: int64

lead_attributes
{'show_contact_an_agent': True}     7915
{'show_contact_an_agent': False}      93
Name: count, dtype: int64

In [ ]:
date = data.drop(['products','lead_attributes'],axis=1)

Given the above statement that the list price is too close to the sold price, it can be used to fill in missing values in the sold price column.

In [68]:
print('missing values in "sold price":',len(data[data['descrip_sold_price'].isna()]))

missing values in "sold price": 1443


In [ ]:
data['descrip_sold_price'] = data['descrip_sold_price'].fillna(data['list_price'])

However, rows with no price info at all will need to be dropped, and we can now drop list_price.

In [70]:
print('no price info:',len(data[(data['descrip_sold_price'].isna()) & (data['list_price'].isna())]))

no price info: 151


In [110]:
# drop rows that still have NaNs 
data = data.dropna(subset=['descrip_sold_price']).drop('list_price',axis=1)

We don't think we are going to be using the price reduction data, and flags_is_price_reduced is broken, showing True _and_ False with numeric values in the price_reduced_amount column, so those columns are being dropped. 

In [71]:
data = data.drop(['price_reduced_amount','flags_is_price_reduced'],axis=1)

The hoa tags also have issues, and are mostly empty, so they are being dropped. 

In [72]:
print('has any hoa tag:',len(data[(data['hoa']>0)|(data['low_hoa']>0)|(data['no_hoa']>0)]))
print('yes hoa AND no hoa:',len(data[(data['hoa']>0) & (data['no_hoa']>0)]))

has any hoa tag: 291
yes hoa AND no hoa: 15


In [73]:
data = data.drop(['hoa','low_hoa','no_hoa'],axis=1)

It is unclear what the floor_plan tag is related to, as all buildings by definition have a floor plan. Given that this tag is apparently optional it cannot be used to draw any conclusions. The open_floor_plan tag only occurs alongside the floor_plan tag, but also probably can't be used to draw conclusions, given the many missing values. A lack of open floor plan tag doesn't necessarily mean the property _isn't_ open plan. We are dropping those columns.

In [74]:
print('floor plan:',len(data[data['floor_plan']>0]))
print('open + floor:',len(data[(data['open_floor_plan']>0) & (data['floor_plan']>0)]))

floor plan: 1057
open + floor: 607


In [75]:
data = data.drop(['floor_plan','open_floor_plan'],axis=1)

We have decided we are not going to work with the tags dealing with tags related to features of the surrounding area unless we have time left over, so those are going to be dropped for now.

In [ ]:
skipped_feat = ['community_outdoor_space','recreation_facilities','rv_or_boat_parking','trails','shopping','waterfront','beach',
'spa_or_hot_tub','horse_facilities','pond','swimming_pool','wooded_land','community_swimming_pool','tennis_court',
'tennis','community_park','clubhouse','park','library','community_center','community_golf','community_tennis_court',
'community_clubhouse','playground','lake','golf_course','volleyball','rv_parking','river_access','basketball_court',
'basketball','community_gym','marina','greenbelt','community_spa_or_hot_tub','community_horse_facilities','soccer',                          
'community_boat_facilities','indoor_basketball_court','baseball','horse_stables','horse_property','equestrian',
'boat_dock','horse_stables','horse_property','equestrian','rv_parking','rv_or_boat_parking','boat_dock',
'community_elevator','elevator','outbuilding','detached_guest_house','guest_house','hill_or_mountain_view','ocean_view',
'city_view','lake_view','river_view','water_view','view','mountain_view','views','golf_course_view','fruit_trees','greenhouse',
'ranch','farm','senior_community','groundscare','maintenance','gated_community','community_security_features','medicalcare',
'disability_features','handicap_access',
]

In [77]:
data = data.drop(skipped_feat,axis=1)

Given the output below, it looks like 'subtype' and 'type' are reversed, so we have decided to group properties first based on 'subtype' and then by type. This will eliminate the tiny categories. 

In [78]:
display(data[['descrip_type','descrip_sub_type']].value_counts())
display(data['descrip_type'].value_counts())

descrip_type                 descrip_sub_type
condos                       condo               842
townhomes                    townhouse           555
apartment                    condo                20
condo_townhome_rowhome_coop  townhouse             5
duplex_triplex               townhouse             5
Name: count, dtype: int64

descrip_type
single_family                  5457
condos                          842
townhomes                       555
multi_family                    543
land                            416
mobile                          198
condo                            69
apartment                        20
other                            15
duplex_triplex                    5
condo_townhome_rowhome_coop       5
Name: count, dtype: int64

We are also going to combine 'condos' into the rest of the 'condo' category because that seems like an error. 

In [79]:
data['property_type'] = ''

In [80]:
# assigning values to the new column based on the relevant type column
# -> based on type:
data.loc[data['descrip_type']=='single_family','property_type'] = 'single_family'
data.loc[data['descrip_type']=='multi_family','property_type'] = 'multi_family'
data.loc[data['descrip_type']=='land','property_type'] = 'land'
data.loc[data['descrip_type']=='mobile','property_type'] = 'mobile'
data.loc[data['descrip_type']=='condo','property_type'] = 'condo'
data.loc[data['descrip_type']=='other','property_type'] = 'other'

# -> based on subtype:
data.loc[data['descrip_sub_type']=='condo','property_type'] = 'condo'
data.loc[data['descrip_sub_type']=='townhouse','property_type'] = 'townhouse'

# the leftovers:
data.loc[data['property_type']=='','property_type'] = 'unknown'

# drop old columns
data = data.drop(['descrip_type','descrip_sub_type'],axis=1)

Where the descrip_stories column is empty we can fill some of them in with the single_story or two_or_more_stories tags. 

In [81]:
print('single story tag, no descrip:',len(data[(data['descrip_stories'].isna())&(data['two_or_more_stories'].isna())&(data['single_story']>0)]))
print('2+ tag, no descrip:',len(data[(data['descrip_stories'].isna())&(data['two_or_more_stories']>0)&(data['single_story'].isna())]))
print('none of them:',len(data[(data['descrip_stories'].isna())&(data['two_or_more_stories'].isna())&(data['single_story'].isna())]))
display(data['descrip_stories'].value_counts())

single story tag, no descrip: 118
2+ tag, no descrip: 154
none of them: 1627


descrip_stories
1.0     3306
2.0     2474
3.0      432
4.0       25
6.0       16
10.0       6
8.0        1
Name: count, dtype: int64

While not perfect, most multistory homes are two stories (84%), so it's reasonably likely that would be the number. <p>
Missing values for descrip stories will be filled with:<br>
1 for available single story tag<br>
2 for available two or more tag<br>
0 for missing all of them

In [82]:
# fill empty values with 0
data.loc[:,'descrip_stories'] = data['descrip_stories'].fillna(0)

# fill 1 for single story using list of indexes
data.loc[((data[(data['descrip_stories']==0)&   # where descrip is empty and
                (data['single_story']>0)        # single story is valid
                ].index).tolist()),'descrip_stories'] = 1

# # fill 2 for multistory using list of indexes
data.loc[((data[(data['descrip_stories']==0)&   # where descrip is empty and
                (data['two_or_more_stories']>0) # multistory is valid
                ].index).tolist()),'descrip_stories'] = 2

# drop redundant columns
data = data.drop(['single_story','two_or_more_stories'],axis=1)

### Dealing with Tags

Consider the fact that with tags, there are a lot of categorical variables.
- How many columns would we have if we OHE tags, city and state?
- Perhaps we can get rid of tags that have a low frequency.

_(could not use .loc to only fillna the expanded tags columns, so those are going to have to be applied case by case.)_

Garage tags are cumulative; anything with a garage_3_or_more tag will also have a garage_1_or_more and garage_2_or_more. These seem to be refering to the total car capacity, as the descrip_garage column can be more than 3, but if the garage_3_or_more tag is present it will be at **least** 3. Some properties with garage_n_or_more tags have blank descrip_garage values, so that should be filled in and used as the only garage column.

In [83]:
display(data.loc[[0,6,244,2136],['garage_1_or_more','garage_2_or_more','garage_3_or_more','descrip_garage']])

print('only descrip filled in:',len(data[(data['garage_1_or_more'].isna())&(data['garage_2_or_more'].isna())&
                                         (data['garage_3_or_more'].isna())&(data['descrip_garage']>0)]))
print('missing descrip:',len(data[(data['descrip_garage'].isna())&((data['garage_1_or_more']>0)|(data['garage_2_or_more']>0)|(data['garage_3_or_more']>0))]))

print('inside the rows missing descrip:')
display(data[((data['garage_1_or_more']>0)|(data['garage_2_or_more']>0)|(data['garage_3_or_more']>0))&(data['descrip_garage'].isna())][['garage_1_or_more','garage_2_or_more','garage_3_or_more']].sum())

,garage_1_or_more,garage_2_or_more,garage_3_or_more,descrip_garage
0,1.0,NaN,NaN,1.0
6,1.0,1.0,NaN,2.0
244,1.0,1.0,1.0,3.0
2136,1.0,1.0,1.0,6.0


only descrip filled in: 38
missing descrip: 339
inside the rows missing descrip:


garage_1_or_more    339.0
garage_2_or_more      0.0
garage_3_or_more      0.0
dtype: float64

Since all the rows with an empty descrip_garage value only have garage_1 tags, fill descip_garage with 1 for those rows. 

In [84]:
# fillna(0) the relevant columns
data.loc[:,'garage_1_or_more'] = data['garage_1_or_more'].fillna(0)
data.loc[:,'descrip_garage'] = data['descrip_garage'].fillna(0)

# using list of indexes to select rows to fill
data.loc[((data[(data['descrip_garage']==0.0)&  # nothing in descrip column and
                (data['garage_1_or_more']>0     # valid value in garage_1 column
                 )].index).tolist()),'descrip_garage'] = 1

# drop the now redundant columns, rename the updated one
data = data.drop(['garage_1_or_more','garage_2_or_more','garage_3_or_more'],axis=1)
data = data.rename(columns={'descrip_garage': 'garage_car_spaces'})

According to Google, forced air and central heat are both types of heating, just with slightly different mechanisms, so those can be combined into one column 'heating' and then dropped.

In [85]:
print('properties with either forced air or central heat:')
print(len(data[(data['forced_air']>0) | (data['central_heat']>0)]))

properties with either forced air or central heat:
4594


In [86]:
# fillna(0) the relevant columns
data.loc[:,'central_heat'] = data['central_heat'].fillna(0)
data.loc[:,'forced_air'] = data['forced_air'].fillna(0)

# create heating column, default value 0
data['heating'] = 0.0

# assign the rows with those tags a 1 using list index
data.loc[((data[(data['central_heat']>0)|   # valid value for central heat or
                (data['forced_air']>0)      # valid value for forced air
                ].index).tolist()),'heating'] = 1.0

#drop old columns
data = data.drop(['central_heat','forced_air'],axis=1)

solar_panels is completely redundant with solar_system and never occurs on its own; renaming solar_system 'solar_power' and dropping solar_panel. 

In [87]:
print('solar system or panels:',len(data[(data['solar_system']>0) | (data['solar_panels']>0)]))
print('both:',len(data[(data['solar_system']>0) & (data['solar_panels']>0)]))
print('only panels:',len(data[(data['solar_system']==0) & (data['solar_panels']>0)]))

solar system or panels: 151
both: 84
only panels: 0


In [88]:
# fillna(0) the relevant column
data.loc[:,'solar_system'] = data['solar_system'].fillna(0)

data = data.rename(columns={'solar_system': 'solar_power'})
data = data.drop('solar_panels',axis=1)

There are four tags for porches, which are going to be combined into one 'porch' column.

In [89]:
# create porch column, default value 0
data['porch'] = 0.0

# assign the rows with those tags a 1 using list of indexes
data.loc[((data[(~data['front_porch'].isna())|      # where front porch isn't null or
                (~data['large_porch'].isna())|      # where large porch isn't null or
                (~data['screen_porch'].isna())|     # where screen porch isn't null or
                (~data['wrap_around_porch'].isna()) # where wrap around porch isn't null
                ].index).tolist()),'porch'] = 1.0

# drop old columns
data = data.drop(['front_porch','large_porch','screen_porch','wrap_around_porch'],axis=1)

While some properties, like apartments, might not have a dedicated laundry room, laundry rooms generally have washers and dryers. So these will be combined into 'laundry_facilities'.

In [90]:
data[(data['washer_dryer']==1.0)|(data['laundry_room']==1.0)][['washer_dryer','laundry_room']].sum()

washer_dryer    2252.0
laundry_room    2248.0
dtype: float64

In [91]:
# create Laundry_facilities column, default value 0
data['laundry_facilities'] = 0.0

# assign the rows with those tags a 1 using list of indexes
data.loc[((data[(~data['washer_dryer'].isna())| # where washer/dryer isn't null or
                (~data['laundry_room'].isna())  # where laundry room isn't null
                ].index).tolist()),'laundry_facilities'] = 1.0

# drop old columns 
data = data.drop(['washer_dryer','laundry_room'],axis=1)

Kitchen and ceiling features:<p>
We are going to handle this in two ways:  the creation of 'notable_kitchen' for any tag pretaining to kitchen, and 'notable_ceiling' for the ceiling ones, and then a 'recently_updated' column with a max value of 2 based on the presence of 'updated_kitchen' and 'new_roof'. Without interior design knowledge there is no way to meaninfully rank the various types of kitchen or ceiling, but their inclusion presumably means they are desirable. <p>
_Note: ceilings are interior and roofs are exterior, so the 'new_roof' column is not part of the notable_ceiling aggregate._

In [92]:
# create new columns, default values all 0
data[['notable_kitchen','notable_ceiling','recently_updated']] = 0.0

# assign each column 1s in the appropriate rows using list of indexes
# -> kitchens 
data.loc[((data[(~data['updated_kitchen'].isna())|  # where updated kitchen isn't null or
                (~data['modern_kitchen'].isna())|   # where modern kitchen isn't null or
                (~data['large_kitchen'].isna())|    # where large kitchen isn't null or
                (~data['granite_kitchen'].isna())|  # where granite kitchen isn't null or
                (~data['open_kitchen'].isna())|     # where open kitchen isn't null or
                (~data['gourmet_kitchen'].isna())|  # where gourmet kitchen isn't null or
                (~data['kitchen_island'].isna())|   # where kitchen island isn't null or
                (~data['outdoor_kitchen'].isna())|  # where outdoor kitchen isn't null or
                (~data['white_kitchen'].isna())|    # where white kitchen isn't null or
                (~data['two_kitchen'].isna())       # where two kitchen isn't null
                ].index).tolist()),'notable_kitchen'] = 1.0

# -> ceilings
data.loc[((data[(~data['high_ceiling'].isna())|         # where high ceiling isn't null or
                (~data['coffer_ceiling'].isna())|       # where coffer ceiling isn't null or
                (~data['cathedral_ceiling'].isna())|    # where cathedral ceiling isn't null or
                (~data['vaulted_ceiling'].isna())       # where vaulted ceiling isn't null 
                ].index).tolist()),'notable_ceiling'] = 1.0

# -> recent updates
# (has both)
data.loc[((data[(~data['new_roof'].isna())&         # where new roof isn't null and
                (~data['updated_kitchen'].isna())   # where updated kitchen isn't null 
                ].index).tolist()),'recently_updated'] = 1.0
# only kitchen
data.loc[((data[(data['new_roof'].isna())&          # where new roof is null and
                (~data['updated_kitchen'].isna())   # where updated kitchen isn't null  
                ].index).tolist()),'recently_updated'] = 1.0
# only roof 
data.loc[((data[(~data['new_roof'].isna())&         # where new roof isn't null and
                (data['updated_kitchen'].isna())    # where updated kitchen isn null 
                ].index).tolist()),'recently_updated'] = 1.0

# drop old columns
data = data.drop(['updated_kitchen','modern_kitchen','large_kitchen','open_kitchen','granite_kitchen','gourmet_kitchen',
            'kitchen_island','outdoor_kitchen','white_kitchen','two_kitchen','high_ceiling','coffer_ceiling',
            'cathedral_ceiling','vaulted_ceiling','new_roof'],axis=1)

We have decided that we are going to consider 'media_room', 'theater_room' and 'game_room' the same for our purposes, as they logically are essentially the same and there are very few of them in the overall dataset. 

In [93]:
# create new column, default values 0
data['multimedia_room'] = 0.0

# assign the rows with those tags a 1 using list of indexes
data.loc[((data[(~data['media_room'].isna())|   # where media room isn't null or
                (~data['theater_room'].isna())| # where theater room isn't null or
                (~data['game_room'].isna())     # where game room isn't null 
                ].index).tolist()),'multimedia_room'] = 1.0

# drop old columns
data = data.drop(['media_room','theater_room','game_room'],axis=1)

Going to collapse fenced_yard, private_backyard and fenced_courtyard into 'enclosed_yard'; given that the courtyard tag only occurs on single family homes, it cannot be talking about the kind of central courtyard that happens on some apartment complexes, and belongs with the others. 

In [94]:
data[data['fenced_courtyard']>0][['fenced_courtyard','property_type']].value_counts()

fenced_courtyard  property_type
1.0               single_family    10
Name: count, dtype: int64

In [95]:
# create new column with default value of 0
data['enclosed_yard'] = 0.0

# assign the rows with those tags a 1 using list of indexes
data.loc[((data[(data['fenced_yard']>0)|   # where fenced yard isn't null or
                (data['private_backyard']>0)| # where private backyard isn't null or
                (data['fenced_courtyard']>0)     # where fenced courtyard isn't null 
                ].index).tolist()),'enclosed_yard'] = 1.0

# drop the old columns
data = data.drop(['fenced_yard','private_backyard','fenced_courtyard'],axis=1)

carport and private_parking are largely redundant with garage information, but not completely. However, there is no way to know how many cars each fits. 

data[data['carport']==1.0][['carport','garage_car_spaces']].value_counts()

data[data['private_parking']==1.0][['private_parking','descrip_garage']]

[['master_bedroom','master_suite','two_master_suites','master_bathroom','first_floor_master_bedroom','dual_master_bedroom']]

There 

In [123]:
data[data['descrip_beds'].isna()][['master_bedroom','master_suite','two_master_suites','master_bathroom','first_floor_master_bedroom','dual_master_bedroom']].value_counts()

Series([], Name: count, dtype: int64)

In [111]:
data.keys()

Index(['carport', 'cul_de_sac', 'family_room', 'hardwood_floors',
       'hill_or_mountain_view', 'basement', 'big_yard', 'efficient',
       'ocean_view', 'rental_property', 'furniture', 'city_view',
       'community_security_features', 'corner_lot', 'lake_view', 'river_view',
       'ensuite', 'private_courtyard', 'central_air', 'dishwasher',
       'energy_efficient', 'fireplace', 'fixer_upper', 'master_bedroom',
       'big_lot', 'master_suite', 'medicalcare', 'disability_features',
       'golf_course_lot_or_frontage', 'water_view', 'maintenance',
       'groundscare', 'jack_and_jill_bathroom', 'two_master_suites',
       'big_bathroom', 'investment_opportunity', 'view', 'den_or_office',
       'beautiful_backyard', 'smart_homes', 'guest_parking', 'storm_shelter',
       'exposed_brick', 'master_bathroom', 'solar_power', 'gated_community',
       'views', 'security', 'private_bathroom', 'fruit_trees', 'mountain_view',
       'farm', 'ranch', 'pets_allowed', 'senior_community', 'c

### Dealing with Cities

- Sales will vary drastically between cities and states.  Is there a way to keep information about which city it is without OHE?
- Could we label encode or ordinal encode?  Yes, but this may have undesirable effects, giving nominal data ordinal values.
- What we can do is use our training data to encode the mean sale price by city as a feature (a.k.a. Target Encoding)
    - We can do this as long as we ONLY use the training data - we're using the available data to give us a 'starting guess' of the price for each city, without needing to encode city explicitly
- If you replace cities or states with numerical values (like the mean price), make sure that the data is split so that we don't leak data into the training selection. This is a great time to train test split. Compute on the training data, and join these values to the test data
- Note that you *may* have cities in the test set that are not in the training set. You don't want these to be NA, so maybe you can fill them with the overall mean

In [97]:
# perform train test split here
# do something with state and city

## Extra Data - STRETCH

> This doesn't need to be part of your Minimum Viable Product (MVP). We recommend you write a functional, basic pipeline first, then circle back and join new data if you have time

> If you do this, try to write your downstream steps in a way it will still work on a dataframe with different features!

- You're not limited to just using the data provided to you. Think/ do some research about other features that might be useful to predict housing prices. 
- Can you import and join this data? Make sure you do any necessary preprocessing and make sure it is joined correctly.
- Example suggestion: could mortgage interest rates in the year of the listing affect the price? 

In [98]:
# import, join and preprocess new data here

## EDA/ Visualization

Remember all of the EDA that you've been learning about?  Now is a perfect time for it!
- Look at distributions of numerical variables to see the shape of the data and detect outliers.    
    - Consider transforming very skewed variables
- Scatterplots of a numerical variable and the target go a long way to show correlations.
- A heatmap will help detect highly correlated features, and we don't want these.
    - You may have too many features to do this, in which case you can simply compute the most correlated feature-pairs and list them
- Is there any overlap in any of the features? (redundant information, like number of this or that room...)

In [99]:
# perform EDA here

## Scaling and Finishing Up

Now is a great time to scale the data and save it once it's preprocessed.
- You can save it in your data folder, but you may want to make a new `processed/` subfolder to keep it organized